In [ ]:
import subprocess
import sys
import boto3
import sagemaker
import os
from dotenv import load_dotenv
from IPython.display import clear_output
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

In [ ]:
# Retrieve the Jupyter Notebook token
jupyter_token = sys.argv[1] if len(sys.argv) > 1 else ""

# Retrieve the AWS region
session = boto3.session.Session()
current_region = session.region_name

# Retrieve the AWS account ID from the IAM user
sts_client = session.client('sts')
response = sts_client.get_caller_identity()
account_id = response['Account']

# Execute the Docker login command
docker_login_command = f"aws ecr get-login-password --region {current_region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{current_region}.amazonaws.com"
subprocess.run(["bash", "-c", docker_login_command], input=jupyter_token.encode("utf-8"))

In [ ]:
# Build the Docker container
docker_build_command = "docker build -t medical-mixtral-7b-v2k ."
subprocess.run(["bash", "-c", docker_build_command])
clear_output()

In [ ]:
# Create a new ECR repository
ecr_repository_name = "aimedical"
create_ecr_repository_command = f"aws ecr create-repository --repository-name {ecr_repository_name} --region {current_region}"
subprocess.run(["bash", "-c", create_ecr_repository_command])

In [ ]:
# Tag the Docker image
tag_docker_image_command = f"docker tag medical-mixtral-7b-v1k:latest {account_id}.dkr.ecr.{current_region}.amazonaws.com/{ecr_repository_name}:latest"
subprocess.run(["bash", "-c", tag_docker_image_command])

In [ ]:
# Push the Docker image to ECR
push_docker_image_command = f"docker push {account_id}.dkr.ecr.{current_region}.amazonaws.com/{ecr_repository_name}:latest"
subprocess.run(["bash", "-c", push_docker_image_command])

In [ ]:
# Get the default SageMaker execution role
role = sagemaker.get_execution_role()
print("SageMaker Execution Role ARN:", role)


In [ ]:
# Check if .env file exists
if not os.path.exists('.env'):
    # Print the URL for Hugging Face token
    print("Please go to the following URL and obtain your Hugging Face token:")
    print("https://huggingface.co/settings/tokens")
    print()
    # Prompt user to enter HF_TOKEN manually
    hf_token = input("Please enter your Hugging Face token: ")

    # Create or append to .env file
    with open('.env', 'a') as f:
        f.write(f"HF_TOKEN={hf_token}\n")

# Load the .env file
load_dotenv()
# Retrieve the value of HF_TOKEN from the environment variables
secret_hf = os.environ.get('HF_TOKEN')
# Clear output to hide the token
from IPython.display import clear_output
clear_output()
# Print the value of HF_TOKEN
print("Loaded HF Token")

In [ ]:
# Set up the environment variables (assuming secret_hf is a valid secret)
environment_variables = {
    'HF_TOKEN': secret_hf
}
instance_type = 'ml.g4dn.xlarge'

# Define the code to run (assuming FineTune.ipynb is within the container)
code_file = 'FineTune.ipynb'
# command=['jupyter', 'nbconvert', '--to', 'notebook', '--execute', 'FineTune.ipynb']

# Set up the ScriptProcessor
script_processor = ScriptProcessor(
    command=['jupyter', 'nbconvert', '--to', 'notebook', '--execute'],
    image_uri=f'{account_id}.dkr.ecr.{current_region}.amazonaws.com/{ecr_repository_name}:latest',
    role=role,
    instance_count=1,
    instance_type=instance_type,
    env=environment_variables
)

# Run the script
script_processor.run(code=code_file)
